# Assign positives and negatives 

In [1]:
import itertools
import random
import sys
import json

import py2neo
import pandas

## Startup neo4j and connections

In [2]:
with open('servers.json') as read_file:
    instances = json.load(read_file)

for instance in instances:
    uri = 'http://localhost:{}/db/data/'.format(instance['port'])
    instance['py2neo'] = py2neo.Graph(uri)
    if instance['name'] == 'rephetio-v2.0':
        neo_unperm = instance['py2neo']

hetnets = [x['name'] for x in instances]
hetnets

['rephetio-v2.0',
 'rephetio-v2.0_perm-1',
 'rephetio-v2.0_perm-2',
 'rephetio-v2.0_perm-3',
 'rephetio-v2.0_perm-4',
 'rephetio-v2.0_perm-5']

## Create partitions

In [3]:
def to_df(record_list):
    """Convert a py2neo RecordList to a dataframe"""
    return pandas.DataFrame(record_list.records, columns = record_list.columns)

In [4]:
compound_df = pandas.read_table('../summary/compounds.tsv')
disease_df = pandas.read_table('../summary/diseases.tsv')

In [5]:
'{} compounds × {} diseases = {} pairs'.format(
    len(disease_df), len(compound_df), len(disease_df) * len(compound_df))

'134 compounds × 1393 diseases = 186662 pairs'

In [6]:
nonzero_prior_pairs = set(itertools.product(
    compound_df.query("treats > 0").compound_id,
    disease_df.query("treats > 0").disease_id)
)

In [7]:
indication_query = '''
MATCH (compound:Compound)-[treatment:TREATS_CtD]->(disease:Disease)
RETURN
  compound.identifier AS compound_id,
  disease.identifier AS disease_id
ORDER BY
  compound_id, disease_id
'''

def partition(neo):
    """
    Extract negative and positive compound-disease pairs from a py2neo.Graph.
    """
    
    # query hetnet for all positive drug-treat-disease examples
    # query hetnet for all potential drug-disease pairs
    # randomly sample 4x the number of non-positive examples for use as negative examples

    
    # Use TREATS_CtD as positives
    indication_df = to_df(neo.cypher.execute(indication_query))
    positives = set(zip(indication_df.compound_id, indication_df.disease_id))

    # Use nonzero-prior pairs excluding non-negatives as negatives
    negatives = nonzero_prior_pairs - positives
    negatives = random.sample(negatives, k=len(positives) * 4)
    rows = list()
    for status, pairs in (0, negatives), (1, positives):
        for drug, disease in pairs:
            rows.append((drug, disease, status))
    df = pandas.DataFrame(rows, columns=['compound_id', 'disease_id', 'status'])
    df = df.sort_values(['disease_id', 'status', 'compound_id'])
    return df

In [8]:
part_dfs = list()
for seed, instance in enumerate(instances):
    random.seed(seed, version=2)
    part_df = partition(instance['py2neo'])
    part_df.insert(0, 'hetnet', instance['name'])
    part_dfs.append(part_df)
part_df = pandas.concat(part_dfs)

In [9]:
part_df.head()

,hetnet,compound_id,disease_id,status
837,rephetio-v2.0,DB00178,DOID:0050741,0
547,rephetio-v2.0,DB00242,DOID:0050741,0
205,rephetio-v2.0,DB00264,DOID:0050741,0
798,rephetio-v2.0,DB00275,DOID:0050741,0
542,rephetio-v2.0,DB00394,DOID:0050741,0


In [10]:
part_df.groupby("hetnet")["status"].value_counts()

hetnet                status
rephetio-v2.0         0         2460
                      1          615
rephetio-v2.0_perm-1  0         2460
                      1          615
rephetio-v2.0_perm-2  0         2460
                      1          615
rephetio-v2.0_perm-3  0         2460
                      1          615
rephetio-v2.0_perm-4  0         2460
                      1          615
rephetio-v2.0_perm-5  0         2460
                      1          615
dtype: int64

In [11]:
## Create an even-matrix for unpermuted observations
template_df = pandas.DataFrame(list(itertools.product(hetnets, compound_df.compound_id)), columns=['hetnet', 'compound_id'])
unperm_pair_df = part_df.query("hetnet == 'rephetio-v2.0'")[['compound_id', 'disease_id']]
unperm_pair_df = template_df.merge(unperm_pair_df)
unperm_pair_df = unperm_pair_df.merge(part_df, how='left')
unperm_pair_df = unperm_pair_df[unperm_pair_df.status.isnull()]
unperm_pair_df.status = unperm_pair_df.status.fillna(0).astype(int)
unperm_pair_df.head(2)

,hetnet,compound_id,disease_id,status
11,rephetio-v2.0_perm-1,DB01048,DOID:0060073,0
12,rephetio-v2.0_perm-1,DB01048,DOID:10283,0


In [12]:
part_df['primary'] = 1
unperm_pair_df['primary'] = 0
full_part_df = pandas.concat([part_df, unperm_pair_df])
full_part_df = full_part_df.sort_values(['compound_id', 'disease_id', 'hetnet'])
full_part_df.head(5)

,hetnet,compound_id,disease_id,status,primary
1752,rephetio-v2.0_perm-2,DB00014,DOID:1024,0,1
2788,rephetio-v2.0,DB00014,DOID:10283,1,1
7917,rephetio-v2.0_perm-1,DB00014,DOID:10283,0,0
7926,rephetio-v2.0_perm-2,DB00014,DOID:10283,0,0
7935,rephetio-v2.0_perm-3,DB00014,DOID:10283,0,0


In [13]:
full_part_df.to_csv('data/partitions.tsv', sep='\t', index=False)

In [14]:
# Number of hetnet-compound-disease pairs
len(full_part_df)

31752

In [15]:
# Number of positives and negatives per purpose
pandas.crosstab(full_part_df.primary, full_part_df.status)

status,0,1
primary,,
0,13302,0
1,14760,3690


In [16]:
# Number of positives and negatives per hetnet
pandas.crosstab(full_part_df.hetnet, full_part_df.status)

status,0,1
hetnet,,
rephetio-v2.0,2460,615
rephetio-v2.0_perm-1,5115,615
rephetio-v2.0_perm-2,5138,615
rephetio-v2.0_perm-3,5114,615
rephetio-v2.0_perm-4,5120,615
rephetio-v2.0_perm-5,5115,615
